In [40]:
from strands import Agent, tool
from strands_tools import calculator, current_time

# Define a custom tool as a Python function using the @tool decorator
@tool
def letter_counter(word: str, letter: str) -> int:
    """
    Count occurrences of a specific letter in a word.

    Args:
        word (str): The input word to search in
        letter (str): The specific letter to count

    Returns:
        int: The number of occurrences of the letter in the word
    """
    if not isinstance(word, str) or not isinstance(letter, str):
        return 0

    if len(letter) != 1:
        raise ValueError("The 'letter' parameter must be a single character")

    return word.lower().count(letter.lower())


In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

agent = Agent(
    model="openai.gpt-oss-120b-1:0",
    tools=[calculator, current_time, letter_counter],
    conversation_manager=SlidingWindowConversationManager(
        window_size=4,  # Maximum number of messages to keep
        should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window
    )
)

In [45]:
message = """
I have 4 requests:

1. What is current year?
2. Calculate 56153750 / 250
3. Tell me how many letter R's are in the word "strawberry and blueberry are berries" 🍓
4. What is the sum of answers from 1, 2, and 3?
"""

In [47]:
result = agent(message)
print(result)

We need current year => use current_time function, parse year. Then calculate division, count letter R (case-insensitive? The phrase contains both 'R' and 'r'. Likely count both upper and lower case? The request says letter R's, probably case-insensitive. Count occurrences of 'r' and 'R'. Let's compute manually.

Phrase: "strawberry and blueberry are berries"

Write it out: s t r a w b e r r y   a n d   b l u e b e r r y   a r e   b e r r i e s

Let's count r's:

"strawberry": letters: s t r a w b e r r y → r appears 1 (third letter) and then double r? Actually "strawberry" has 'r' after 't', then later "rr"? No, "strawberry": s t r a w b e r r y => r at position3, then 'r' near end? There's "rr"? Actually "berry" part has 'r', 'r'? "berry" is b e r r y, so there are two r's there. So total in "strawberry": 1 (in 'straw') + 2 (in 'berry') =3.

"and": none.

"blueberry": b l u e b e r r y => two r's in 'berry'. So 2.

"are": a r e => 1.

"berries": b e r r i e s => two r's. So 2.

Now s

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 56153750 / 250      │                                                                            │
│  │ Result    │ 224615              │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 56153750 / 250      │                                                                            │
│  │ Result    │ 224615              │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

We have the current time from tool: 2025-12-16. Year is 2025. Division result is 224615. Letter count is 8. Now sum of answers from 1,2,3: 2025 + 224615 + 8 = 226648. Need to present answers to each request. Provide numbers.**1. Current year:** 2025  

**2. 56153750 ÷ 250 =** 224 615  

**3. Number of “r” letters in “strawberry and blueberry are berries”:** 8  

**4. Sum of the three answers:** 2025 + 224 615 + 8 = **226 648****1. Current year:** 2025  

**2. 56153750 ÷ 250 =** 224 615  

**3. Number of “r” letters in “strawberry and blueberry are berries”:** 8  

**4. Sum of the three answers:** 2025 + 224 615 + 8 = **226 648**



In [48]:
print(result.metrics.get_summary())

{'total_cycles': 4, 'total_duration': 1.371575117111206, 'average_cycle_time': 0.3428937792778015, 'tool_usage': {'current_time': {'tool_info': {'tool_use_id': 'tooluse_q3gIyKDjRHqOqWhZsS1N8Q', 'name': 'current_time', 'input_params': {'timezone': 'UTC'}}, 'execution_stats': {'call_count': 1, 'success_count': 1, 'error_count': 0, 'total_time': 0.0005462169647216797, 'average_time': 0.0005462169647216797, 'success_rate': 1.0}}, 'letter_counter': {'tool_info': {'tool_use_id': 'tooluse_rPpmu9TETL2lPczS3IzhSQ', 'name': 'letter_counter', 'input_params': {'word': 'strawberry and blueberry are berries', 'letter': 'r'}}, 'execution_stats': {'call_count': 2, 'success_count': 2, 'error_count': 0, 'total_time': 0.01452779769897461, 'average_time': 0.007263898849487305, 'success_rate': 1.0}}, 'calculator': {'tool_info': {'tool_use_id': 'tooluse_ZHqk7a_JQiytWv77ZdZ3zQ', 'name': 'calculator', 'input_params': {'mode': 'evaluate', 'expression': '56153750 / 250', 'precision': 10}}, 'execution_stats': {'

In [51]:
print(agent.conversation_manager.get_state())
state = agent.conversation_manager.get_state()

{'__name__': 'SlidingWindowConversationManager', 'removed_message_count': 0}


In [52]:
import json
print(json.dumps(agent.messages, indent=2))

[
  {
    "role": "user",
    "content": [
      {
        "text": "\nI have 4 requests:\n\n1. What is current year?\n2. Calculate 56153750 / 250\n3. Tell me how many letter R's are in the word \"strawberry and blueberry are berries\" \ud83c\udf53\n4. What is the sum of answers from 1, 2, and 3?\n"
      }
    ]
  },
  {
    "role": "user",
    "content": [
      {
        "text": "\nI have 4 requests:\n\n1. What is current year?\n2. Calculate 56153750 / 250\n3. Tell me how many letter R's are in the word \"strawberry and blueberry are berries\" \ud83c\udf53\n4. What is the sum of answers from 1, 2, and 3?\n"
      }
    ]
  },
  {
    "role": "assistant",
    "content": [
      {
        "reasoningContent": {
          "reasoningText": {
            "text": "We need current year => use current_time function, parse year. Then calculate division, count letter R (case-insensitive? The phrase contains both 'R' and 'r'. Likely count both upper and lower case? The request says letter R's, p

In [53]:
result = agent("summarize the previous conversation in one sentence")
print(result)

We need to summarize: user asked 4 requests, assistant gave year, division, count, sum. Summarize in one sentence.The assistant provided the current year (2025), calculated 56153750 ÷ 250 = 224 615, counted eight “r” letters in “strawberry and blueberry are berries,” and summed these results to 226 648.The assistant provided the current year (2025), calculated 56153750 ÷ 250 = 224 615, counted eight “r” letters in “strawberry and blueberry are berries,” and summed these results to 226 648.



In [54]:
print(json.dumps(agent.messages, indent=2))

[
  {
    "role": "assistant",
    "content": [
      {
        "reasoningContent": {
          "reasoningText": {
            "text": "We need current year from time: 2025. Next compute 56153750 / 250. We'll use calculator."
          }
        }
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_bj2hosg8RcOKp-EqH1WYlg",
          "name": "calculator",
          "input": {
            "mode": "evaluate",
            "expression": "56153750 / 250",
            "precision": 10
          }
        }
      },
      {
        "reasoningContent": {
          "reasoningText": {
            "text": "Now count letter R's (case-insensitive?). The word phrase: \"strawberry and blueberry are berries\". Count letter 'R' (both lower and upper?). Phrase has only lowercase letters. Count 'r's. Let's compute manually or use function letter_counter."
          }
        }
      },
      {
        "toolUse": {
          "toolUseId": "tooluse_xlmJyFGsR7Khc1a55C5kUA",
          "name": 

In [55]:
result = agent("what is the difference between  RGB and CMYK color models? anwer in one sentence")
print(result)
print(json.dumps(agent.messages, indent=2))

Need to answer in one sentence describing difference.RGB mixes red, green, and blue light for additive color on screens, while CMYK combines cyan, magenta, yellow, and black inks for subtractive color in printed media.RGB mixes red, green, and blue light for additive color on screens, while CMYK combines cyan, magenta, yellow, and black inks for subtractive color in printed media.

[
  {
    "role": "assistant",
    "content": [
      {
        "reasoningContent": {
          "reasoningText": {
            "text": "We have the current time from tool: 2025-12-16. Year is 2025. Division result is 224615. Letter count is 8. Now sum of answers from 1,2,3: 2025 + 224615 + 8 = 226648. Need to present answers to each request. Provide numbers."
          }
        }
      },
      {
        "text": "**1. Current year:**\u202f2025  \n\n**2.\u202f56153750\u202f\u00f7\u202f250 =**\u202f224\u202f615  \n\n**3. Number of \u201cr\u201d letters in \u201cstrawberry and blueberry are berries\u201d:**\u2

In [56]:
result = agent("Explain the transformer architecture in simple terms, in 3 sentences.")
print(result)
print(json.dumps(agent.messages, indent=2))

User wants explanation of transformer architecture in simple terms, in 3 sentences. We need to produce exactly 3 sentences, simple terms. Let's craft: "A transformer processes data all at once using self‑attention, letting each word look at every other word to understand context." "It stacks layers of attention and feed‑forward networks, each refining the representation." "Because it can be parallelized and captures long‑range relationships, it powers modern language models." That's three sentences.A transformer reads an entire sentence at once and lets each word “pay attention” to every other word, so it can understand the context of each word based on the whole sequence. It does this repeatedly in stacked layers that combine attention with simple feed‑forward neural networks, gradually refining the representation of each word. Because the attention operations can run in parallel and capture long‑range relationships, transformers become extremely effective for modern language and visi